In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
from itertools import groupby
from datetime import datetime
import math

import seaborn as sns
import os

import folium

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 300

In [3]:
import geopandas
import geopy

In [4]:
df=pd.read_csv('../data/interim/clustering_model_input.csv', header=0)
df.head(2)

,property_href,sale_price,full_add,zip_5,beds,baths,lot_size,prop_size,age
0,https://www.redfin.com/NJ/Somerset/48-S-Dover-Ave-08873/home/37840770,225000.0,48 Dover Ave Franklin NJ 08873,8873,3.0,2.0,16000.0,2417.0,40.0
1,https://www.redfin.com/NJ/Somerset/14-Almond-Dr-08873/home/37859940,230000.0,14 Almond Dr Franklin NJ 08873,8873,2.0,1.5,1080.0,1104.0,36.0


In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="tailorbird")
##location = geolocator.geocode("45 Brunswick Ave Metuchen NJ 08840")
##print(location.address)

In [6]:
from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(geolocator.geocode)

df['location'] = df['full_add'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

##df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('5 Smithwold Rd Somerset NJ 08873',), **{}).
Traceback (most recent call last):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self

RateLimiter swallowed an error after 2 retries. Called with (*('5 Smithwold Rd Somerset NJ 08873',), **{}).
Traceback (most recent call last):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._m

RateLimiter caught an error, retrying (1/2 tries). Called with (*('10 Overbrook Rd Franklin NJ 08873',), **{}).
Traceback (most recent call last):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = sel

In [6]:
df.head(2)

,property_href,sale_price,full_add,zip_5,beds,baths,lot_size,prop_size,age,location,point
0,https://www.redfin.com/NJ/Somerset/48-S-Dover-Ave-08873/home/37840770,225000.0,48 Dover Ave Franklin NJ 08873,8873,3.0,2.0,16000.0,2417.0,40.0,None,None
1,https://www.redfin.com/NJ/Somerset/14-Almond-Dr-08873/home/37859940,230000.0,14 Almond Dr Franklin NJ 08873,8873,2.0,1.5,1080.0,1104.0,36.0,"(14, Almond Drive, Ukrainian Village, Franklin Township, Somerset County, New Jersey, 08873, United States of America, (40.50833363472449, -74.50636016742878))","(40.50833363472449, -74.50636016742878, 0.0)"


In [7]:
df.isnull().sum()

property_href      0
sale_price         0
full_add           0
zip_5              0
beds               0
baths              0
lot_size           0
prop_size          0
age                0
location         176
point            176
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [10]:
df.head(2)

,property_href,sale_price,full_add,zip_5,beds,baths,lot_size,prop_size,age,location,point,latitude,longitude,altitude
1,https://www.redfin.com/NJ/Somerset/14-Almond-Dr-08873/home/37859940,230000.0,14 Almond Dr Franklin NJ 08873,8873,2.0,1.5,1080.0,1104.0,36.0,"(14, Almond Drive, Ukrainian Village, Franklin Township, Somerset County, New Jersey, 08873, United States of America, (40.50833363472449, -74.50636016742878))","(40.50833363472449, -74.50636016742878, 0.0)",40.508334,-74.506360,0.0
2,https://www.redfin.com/NJ/Somerset/160-Jefferson-St-08873/home/37845159,230000.0,160 Jefferson St Franklin NJ 08873,8873,2.0,1.0,5001.0,1365.0,63.0,"(160, Jefferson Street, Franklin Township, Somerset County, New Jersey, 08873, United States of America, (40.49460388461539, -74.4664406923077))","(40.49460388461539, -74.4664406923077, 0.0)",40.494604,-74.466441,0.0


In [13]:
df.to_csv("../data/interim/clustering_model_input_with_loc.csv", header=True, index=False)

In [14]:
df.zip_5.value_counts()

8873    222
8807    167
8844    147
7920     46
8876     46
8540     36
7059     26
8805     15
8835     11
8502      9
8558      1
Name: zip_5, dtype: int64